# HW5

For this homework, we are going to work with [*Indoor User Movement Prediction from RSS data*](https://archive.ics.uci.edu/ml/datasets/Indoor+User+Movement+Prediction+from+RSS+data) dataset from UCI.  The homework is due Friday, December 21st midnight. 

## Task 1

Download the dataset and unzip it in under a subdirectory of `data` folder named `rss_data`.

The files we are interested is in the subfolder `dataset`.  Each of these files whose names that start with `MovementAAL_RSS_` contain data collected from indivuduals. Each of these files represent a single data point.  There are 314 of these files, and hence, you have 314 data points.  Each file has 4 columns but the number of rows change from file to file.  

There is also a file named `MovementALL_target.csv` in that folder. This file tells us the class each of these measurements are assigned. Some of these measurements are labelled with +1 and some are labelled with -1.

## Task 2

Construct a SVM model that separates +1 labelled data points from -1 data points.  You must first solve the problem that these datapoints do not have the same number of rows even though they all have the same number of columns. 

## Task 3

Using [Keras](https://keras.io/getting-started/sequential-model-guide/) write a neural network model that separates +1 labelled data points from -1 data points.

## Notes

1. You must document each step of your tasks: what are you doing, why are you doing it, what problems you encountered and how you solved it.  All of these must be explained and documented.  Solutions without sufficient documentations will be penalized accordingly. 50% of your points will come from your code, while the other 50% will come from your explanations.

1. You can use MS Excel to inspect the files, but loading them up to python using pandas and inspecting them there under jupyter is easier.

3. Put the data in a separate subfolder of your `data` folder and rename it `rss_data`. I'll take points off if the data is not saved under the correct place.

1. For both of Task 2 and Task 3, you must split your data into a train and test set, and then evaluate the accuracy of your model on the test set.



# TASK 1

In [2]:
import glob
import pandas as pd
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import neighbors
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

C:\Users\Halil\anakonda3-1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Kütüphaneler yüklendi.

In [3]:
csv_files=glob.glob("..//data//rss_data//dataset//MovementAAL_RSS_*.csv")
frames=[]
for csv in csv_files:
    df=pd.read_csv(csv)
    frames.append(df)

314 Adet veri noktasını glob fonksiyonunu kullanarak for döngüsüyle csv_files listesine atadık sonrasında frames listesine gönderdik.

In [4]:
a=100
for i in frames:
    a=min(a,len(i))
print(a)

19


Dataların feature sayısı eşit değil yani satır sayısı eşit değil. Datanın en sonundan itibaren keseceğizki bulunduğu konumu tespit edelim.

In [5]:

l=[]
for i in frames:
    for row in range(1,20):
        for col in range(4):
            l.append(i.iloc[-row,col])

l isminde bir yatayda bir matris oluşturup düz bir liste haline getiriyorum.

In [7]:
k=np.array(l)
k=k.reshape(314,76)
df=pd.DataFrame(k)

In [8]:
targets = pd.read_csv('..//data//rss_data//dataset//MovementAAL_target.csv')
targets=targets.drop("#sequence_ID",axis=1)

In [12]:
df=np.column_stack([df,targets])
df=pd.DataFrame(df)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,-0.14286,-0.60,-0.28571,-0.10,0.00000,0.04,-0.047619,-0.05,0.00000,0.04,...,-0.047619,0.04,-0.095238,0.05,-0.095238,-0.08,0.142860,-0.55,1.0,1.0
1,0.19048,0.04,-0.38095,0.20,0.19048,0.00,-0.190480,-0.25,0.14286,-0.04,...,0.000000,-0.12,0.142860,0.10,-0.285710,-0.24,0.095238,0.15,1.0,1.0
2,0.28571,0.56,-0.57143,-0.75,0.33333,0.40,-0.952380,-0.65,0.14286,0.56,...,0.428570,0.04,-0.619050,-0.70,0.047619,0.32,-0.714290,-0.30,1.0,1.0
3,0.28571,0.20,-1.00000,-0.70,0.00000,0.04,-1.000000,-0.55,0.00000,0.04,...,0.523810,0.60,-0.428570,-0.55,0.476190,0.32,-0.380950,-0.30,1.0,1.0
4,0.42857,0.68,-0.28571,-0.30,0.42857,0.76,-0.476190,-0.60,0.00000,0.36,...,0.285710,0.60,-0.571430,-1.00,0.142860,0.76,-0.666670,-0.55,1.0,1.0


Dataframe haline getirildi.

# TASK 2

Task 2'yi çözebilmek için datayı 2'ye ayırmak durumundayız. Bunlara train ve test set adlarını verdim. SVM'i uygulamak için çekirdeklerinden biri Radial basis function (RBF) yi kullandım.

In [17]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,:76],df.iloc[:,76])

sv=SVC(kernel="rbf",gamma=0.03,C=2)
sv.fit(X_train,y_train)
predicted=sv.predict(X_test)

print(accuracy_score(y_test,predicted))
print(confusion_matrix(y_test,predicted))

0.6582278481012658
[[28 14]
 [13 24]]


Çıkan sonuçtan anlayabileceğimiz üzere iyi bir sonuç vermedi 0.60~0.70 arasında kaldı.